# AgriAutoML Pipeline Execution in Vertex AI

This notebook demonstrates how to execute the AgriAutoML pipeline directly in Vertex AI Studio.

In [ ]:
# Install required packages
!pip install google-cloud-aiplatform kfp google-cloud-storage pandas pillow numpy

In [ ]:
import os
from google.cloud import aiplatform
from google.cloud import storage
from google.auth import default
from datetime import datetime

# Get default credentials and project
credentials, project_id = default()

# Configuration
REGION = "us-central1"
BUCKET_NAME = "qwiklabs-gcp-00-ffe6db11d36b-bucket"
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root"

# Initialize Vertex AI with default credentials
aiplatform.init(
    project=project_id,
    location=REGION,
    credentials=credentials
)

In [ ]:
def create_sample_data():
    """Create and upload sample datasets to GCS"""
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    
    # Create sample CSV data
    sample_csv = """date,location,crop_type,field_size,rainfall,temperature,yield
2025-01-01,Iowa,corn,5.0,750,25,150
2025-01-15,Kansas,wheat,3.5,500,22,120
2025-02-01,Nebraska,soybean,4.2,600,24,130"""
    
    # Upload tabular data
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    tabular_blob = bucket.blob(f"data/{timestamp}/crop_data.csv")
    tabular_blob.upload_from_string(sample_csv)
    
    print(f"Uploaded sample data to: gs://{BUCKET_NAME}/data/{timestamp}/")
    return f"gs://{BUCKET_NAME}/data/{timestamp}/crop_data.csv"

# Create sample dataset
dataset_uri = create_sample_data()
print(f"Dataset URI: {dataset_uri}")

In [ ]:
from kfp import dsl
from google.cloud import aiplatform

@dsl.pipeline(
    name='AgriAutoML Pipeline',
    description='End-to-end pipeline for agricultural yield prediction'
)
def agri_automl_pipeline(
    project_id: str,
    region: str,
    bucket_name: str,
    vision_dataset_uri: str,
    tabular_dataset_uri: str,
    min_accuracy: float = 0.8
):
    # Import components
    preprocess_op = components.load_component('preprocess')
    train_vision_op = components.load_component('train_vision')
    train_tabular_op = components.load_component('train_tabular')
    deploy_op = components.load_component('deploy')

    # Data preprocessing
    preprocess = preprocess_op(
        vision_data=vision_dataset_uri,
        tabular_data=tabular_dataset_uri,
        bucket_name=bucket_name
    )

    # Train vision model
    train_vision = train_vision_op(
        project_id=project_id,
        region=region,
        dataset_uri=preprocess.outputs['vision_dataset'],
        min_accuracy=min_accuracy
    )

    # Train tabular model
    train_tabular = train_tabular_op(
        project_id=project_id,
        region=region,
        dataset_uri=preprocess.outputs['tabular_dataset'],
        min_accuracy=min_accuracy
    )

    # Deploy models
    deploy = deploy_op(
        project_id=project_id,
        region=region,
        vision_model=train_vision.outputs['model'],
        tabular_model=train_tabular.outputs['model']
    )


In [ ]:
# Create and run pipeline job
job = aiplatform.PipelineJob(
    display_name="agri-automl-pipeline",
    template_path="agri_automl_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        'project_id': project_id,
        'region': REGION,
        'dataset_uri': dataset_uri,
        'min_accuracy': 0.7
    }
)

job.run(sync=True)

In [ ]:
def get_prediction(instance):
    """Get prediction from the deployed model"""
    endpoints = aiplatform.Endpoint.list()
    endpoint = endpoints[0]  # Get the most recently created endpoint
    
    prediction = endpoint.predict([instance])
    return prediction

# Test prediction
test_instance = {
    'date': '2025-03-01',
    'location': 'Iowa',
    'crop_type': 'corn',
    'field_size': 4.5,
    'rainfall': 700,
    'temperature': 23
}

prediction = get_prediction(test_instance)
print(f"Predicted yield: {prediction}")